In [1]:
import os
# ptb 모듈 경로 추가
import sys
sys.path.append('C:/Users/KimDongyoung/Desktop/Github/my_git/mygit/DEEPLEARNING/밑바닥부터시작하는딥러닝2')

In [2]:
import numpy as np
from common.layers import Softmax
from common.time_layers import *
from ch07.seq2seq import Encoder, Seq2seq
from common.base_model import BaseModel
import pickle
from common.util import to_cpu, to_gpu

In [3]:
np.random.seed(3)

T, H = 5, 4 # T: 시계열의 길이, H: 은닉 상태의 차원 수
hs = np.random.randn(T, H) # Encoder에서 출력된 각 시각의 은닉 상태
print(hs)
a = np.array([0.8, 0.1, 0.03, 0.05, 0.02]) # 가중치

ar = a.reshape(5, 1).repeat(4, axis=1)
print(ar.shape)

t = hs * ar
print(t.shape)
print(t)

c = np.sum(t, axis=0)
print(c) # 맥락 벡터


[[ 1.78862847  0.43650985  0.09649747 -1.8634927 ]
 [-0.2773882  -0.35475898 -0.08274148 -0.62700068]
 [-0.04381817 -0.47721803 -1.31386475  0.88462238]
 [ 0.88131804  1.70957306  0.05003364 -0.40467741]
 [-0.54535995 -1.54647732  0.98236743 -1.10106763]]
(5, 4)
(5, 4)
[[ 1.43090278e+00  3.49207880e-01  7.71979745e-02 -1.49079416e+00]
 [-2.77388203e-02 -3.54758979e-02 -8.27414815e-03 -6.27000677e-02]
 [-1.31454507e-03 -1.43165409e-02 -3.94159426e-02  2.65386714e-02]
 [ 4.40659021e-02  8.54786532e-02  2.50168211e-03 -2.02338707e-02]
 [-1.09071990e-02 -3.09295463e-02  1.96473487e-02 -2.20213526e-02]]
[ 1.43500812  0.35396455  0.05165691 -1.56921078]


In [4]:
# 미니배치 버전
np.random.seed(3)

N, T, H = 10, 5, 4
hs = np.random.randn(N, T, H) # 미니배치 수: N , 시계열의 길이: T, 은닉 상태의 차원 수: H -> 하나의 미니배치에 10개의 시계열 데이터가 있고 이 시계열 데이터의 길이는 5이며, 각 시계열 데이터의 은닉 상태의 차원 수는 4이다.
print(hs)

a = np.random.randn(N, T)
ar = a.reshape(N, T, 1).repeat(H, axis=2)
print(ar.shape)
print(ar)

t = hs * ar
print(t.shape)

c = np.sum(t, axis=1)
print(c.shape) # 맥락 벡터
print(c)

[[[ 1.78862847e+00  4.36509851e-01  9.64974681e-02 -1.86349270e+00]
  [-2.77388203e-01 -3.54758979e-01 -8.27414815e-02 -6.27000677e-01]
  [-4.38181690e-02 -4.77218030e-01 -1.31386475e+00  8.84622380e-01]
  [ 8.81318042e-01  1.70957306e+00  5.00336422e-02 -4.04677415e-01]
  [-5.45359948e-01 -1.54647732e+00  9.82367434e-01 -1.10106763e+00]]

 [[-1.18504653e+00 -2.05649899e-01  1.48614836e+00  2.36716267e-01]
  [-1.02378514e+00 -7.12993200e-01  6.25244966e-01 -1.60513363e-01]
  [-7.68836350e-01 -2.30030722e-01  7.45056266e-01  1.97611078e+00]
  [-1.24412333e+00 -6.26416911e-01 -8.03766095e-01 -2.41908317e+00]
  [-9.23792022e-01 -1.02387576e+00  1.12397796e+00 -1.31914233e-01]]

 [[-1.62328545e+00  6.46675452e-01 -3.56270759e-01 -1.74314104e+00]
  [-5.96649642e-01 -5.88594380e-01 -8.73882298e-01  2.97138154e-02]
  [-2.24825777e+00 -2.67761865e-01  1.01318344e+00  8.52797841e-01]
  [ 1.10818750e+00  1.11939066e+00  1.48754313e+00 -1.11830068e+00]
  [ 8.45833407e-01 -1.86088953e+00 -6.028851

### WeightSum 계층    

In [5]:
class WeightSum:
  def __init__(self):
    self.params, self.grads = [], []
    self.cache = None
    
  def forward(self, hs, a):  # hs: Encoder LSTM 계층에서 출력된 각 시각에서의 은닉 상태들, a: 각 시각의 가중치
    N, T, H = hs.shape       # N: 미니배치 크기, T: 시계열 데이터의 수, H: 은닉 상태의 차원 수

    ar = a.reshape(N, T, 1).repeat(H, axis=2)
    t = hs * ar             # 원소 곱 계산, 은닉 상태 hs의 각 행에 가중치 a의 원소를 곱한다.
    c = np.sum(t, axis=1)   # 맥락 벡터

    self.cache = (hs, ar)   # 역전파 계산 시 사용하기 위해 저장
    return c
  
  def backward(self, dc):   # dc: 덧셈 노드로부터 전해지는 미분
    hs, ar = self.cache
    N, T, H = hs.shape

    dt = dc.reshape(N, 1, H).repeat(T, axis=1)  # sum 노드 역전파
    dar = dt * hs
    dhs = dt * ar
    da = np.sum(dar, axis=2)

    return dhs, da

In [6]:
# 가중치 a를 구하는 단계
N, T, H = 10, 5, 4
hs = np.random.randn(N, T, H)
h = np.random.randn(N, H)
hr = h.reshape(N, 1, H).repeat(T, axis=1)

t = hs * hr
print(t.shape)
print(t)

s = np.sum(t, axis=2)
print(s.shape)
print(s)

softmax = Softmax()
a = softmax.forward(s)
print(a.shape)
print(a)

(10, 5, 4)
[[[ 4.41149179e-01  6.66490002e-01 -5.51332190e-01  6.81004293e-02]
  [-2.34006190e-01  4.40872819e-02 -6.16411348e-01 -1.67675428e-01]
  [ 6.95772253e-01 -1.77264522e-01 -5.20578277e-02  3.36974482e-01]
  [-6.08913406e-02 -2.50112901e-01  3.38525278e-01  4.16763579e-01]
  [-3.64282409e-01 -2.16320045e-01  1.17966819e-01 -2.96546825e-01]]

 [[-4.39972084e-01 -1.11426307e+00 -6.61208332e-01 -3.26909444e-01]
  [-2.87204562e-01  1.02766159e-01 -3.69359904e-01  1.18867140e+00]
  [-3.08593208e-02 -8.02448648e-01 -1.15422992e-01  3.00453093e-02]
  [-2.54619783e-01 -1.06988517e+00 -7.51008923e-01 -1.47551425e+00]
  [ 8.60119707e-02 -1.91248664e-02 -1.17422798e-01 -8.05393101e-01]]

 [[-1.02351960e+00  1.76707033e+00  3.40282468e-01 -1.97654264e+00]
  [ 6.00428458e-01  2.40234289e+00 -2.00705490e-01 -2.15682510e+00]
  [-8.11121985e-01 -1.33111573e+00  5.69642692e-01 -1.91709223e+00]
  [ 3.30427085e-01 -2.05161036e+00  4.70008883e-01 -2.77931326e+00]
  [ 2.66678737e-02 -5.47192541e-0

### AttentionWeight 계층    

In [7]:
class AttentionWeight:
  def __init__(self):
    self.params, self.grads = [], []
    self.softmax = Softmax()
    self.cache = None
    
  def forward(self,hs,h): # hs: encoder의 출력(각 시각에서 출력된 은닉 상태), h: decoder의 출력
    N, T, H = hs.shape

    # hs와 h의 내적
    hr = h.reshape(N, 1, H).repeat(T, axis=1)  
    t = hs * hr
    # s가 내적 결과 (N, T) N은 미니배치의 수, T는 시계열의 길이, 만약 N =10, T=5라면 s는 (10, 5)의 형상을 가진다.
    s = np.sum(t, axis=2)

    a = self.softmax.forward(s)
    self.cache = (hs, hr)
    return a
  
  def backward(self, da): # da: softmax의 역전파 출력
    hs, hr = self.cache
    N, T, H = hs.shape

    ds = self.softmax.backward(da)
    dt = ds.reshape(N, T, 1).repeat(H, axis=2)
    dhs = dt * hr
    dhr = dt * hs
    dh = np.sum(dhr, axis=1)

    return dhs, dh

### Attention 계층  

In [8]:
class Attention:
  def __init__(self):
    self.params, self.grads = [], []
    self.attention_weight_layer = AttentionWeight()
    self.weight_sum_layer = WeightSum()
    self.attention_weight = None
    
  def forward(self, hs, h):
    a = self.attention_weight_layer.forward(hs, h) # 가중치 a를 구한다.
    out = self.weight_sum_layer.forward(hs, a)     # 맥락 벡터를 구한다.    
    self.attention_weight = a
    return out
  
  def backward(self, dout):
    dhs0, da = self.weight_sum_layer.backward(dout)
    dhs1, dh = self.attention_weight_layer.backward(da)
    dhs = dhs0 + dhs1
    return dhs, dh

### Time Attention 계층  

In [9]:
class TimeAttention:
  def __init__(self):
    self.params, self.grads = [], []
    self.layers = None
    self.attention_weights = None
    
  # 순전파
  def forward(self, hs_enc, hs_dec): # hs_enc: Encoder의 출력, hs_dec: Decoder의 은닉 상태
    N, T, H = hs_dec.shape           # N: 미니배치 크기, T: 시계열 데이터의 수, H: 은닉 상태의 차원 수
    out = np.empty_like(hs_dec)
    self.layers = []
    self.attention_weights = [] # Attention의 가중치를 저장할 리스트
    
    for t in range(T):      # 시계열의 길이만큼 반복  
      layer = Attention()   # Attention 계층 생성
      out[:, t, :] = layer.forward(hs_enc, hs_dec[:, t, :])
      self.layers.append(layer)
      self.attention_weights.append(layer.attention_weight)
      
    return out
  
  # 역전파
  def backward(self, dout):
    N, T, H = dout.shape
    dhs_enc = 0
    dhs_dec = np.empty_like(dout)
    
    for t in range(T):
      layer = self.layers[t]
      dhs, dh = layer.backward(dout[:, t, :])
      dhs_enc += dhs
      dhs_dec[:, t, :] = dh
      
    return dhs_enc, dhs_dec
  

### Attention을 갖춘 seq2seq2 구현  

In [10]:
class AttentionEncoder(Encoder):
  def forward(self, xs):
    xs = self.embed.forward(xs)
    hs = self.lstm.forward(xs)
    return hs

  def backward(self, dhs):
    dout = self.lstm.backward(dhs)
    self.embed.backward(dout)
    return dout

In [11]:
class AttentionDecoder:
  def __init__(self, vocab_size, wordvec_size, hidden_size):
    V, D, H = vocab_size, wordvec_size, hidden_size
    rn = np.random.randn
    # 가중치 초기화
    # Embedding 계층의 가중치 초기화
    embed_W = (rn(V, D) / 100).astype('f')
    
    # LSTM 계층의 가중치 초기화
    lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
    lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
    lstm_b = np.zeros(4 * H).astype('f')
    
    # Affine 계층의 가중치 초기화
    affine_W = (rn(2*H, V) / np.sqrt(2*H)).astype('f')
    affine_b = np.zeros(V).astype('f')

    # 계층 생성
    self.embed = TimeEmbedding(embed_W)
    self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True)
    self.attention = TimeAttention()
    self.affine = TimeAffine(affine_W, affine_b)
    
    # layers에 생성한 계층을 리스트로 저장
    layers = [self.embed, self.lstm, self.attention, self.affine]

    # 매개변수, 기울기를 각 리스트에 저장
    self.params, self.grads = [], []
    for layer in layers:
      self.params += layer.params
      self.grads += layer.grads

  # 순전파
  def forward(self, xs, enc_hs): # xs: Decoder의 입력 데이터, enc_hs: Encoder의 출력(은닉 상태)
    # Encoder에서의 마지막 은닉 상태를 h에 저장
    h = enc_hs[:, -1]
    self.lstm.set_state(h)

    out = self.embed.forward(xs)
    dec_hs = self.lstm.forward(out)             # des_hs: Decoder LSTM 계층을 통과한 후의 출력 값(은닉 상태)
    c = self.attention.forward(enc_hs, dec_hs)  # c: 맥락 벡터, Encoder의 출력과 Decoder의 출력을 Attention 계층에 통과시켜 맥락 벡터를 구한다.
    out = np.concatenate((c, dec_hs), axis=2)   # concatencate를 통해 맥락 벡터 c와 Decoder의 출력을 결합한다.
    score = self.affine.forward(out)            # score: Affine 계층을 통과한 후의 출력 값

    return score

  def backward(self, dscore):                   # dscore: Softmax with loss 계층의 역전파 출력
    dout = self.affine.backward(dscore)         # dout: Affine 계층의 역전파 출력
    N, T, H2 = dout.shape
    H = H2 // 2

    dc, ddec_hs0 = dout[:, :, :H], dout[:, :, H:] # dc: Affine 계층의 역전파 출력 중 앞부분은 맥락 벡터 c에 대한 기울기, ddec_hs0: LSTM Decoder의 출력에 대한 기울기
    denc_hs, ddec_hs1 = self.attention.backward(dc) # denc_hs: Encoder의 출력에 대한 기울기, ddec_hs1: LSTM Decoder의 출력에 대한 기울기
    ddec_hs = ddec_hs0 + ddec_hs1  # ddec_hs: Affine 계층의 역전파 출력 중 LSTM Decoder의 출력에 대한 기울기와와 Attention 계층의 역전파 출력 중중 LSTM Decoder의 출력에 대한 기울기를 더한 값이다.
    dout = self.lstm.backward(ddec_hs)            # Decoder의 LSTM 계층에 입력한 x 값에 대한 기울기  
    dh = self.lstm.dh                             # dh: Encoder의 마지막 은닉 상태에 대한 기울기
    denc_hs[:, -1] += dh                          # Encoder의 마지막 은닉 상태에 대한 기울기를 갱신한다. 
    self.embed.backward(dout)                     # Decoder의 Embedding 계층에 대한 기울기

    return denc_hs                                # Encoder의 은닉 상태에 대한 기울기를 반환 
  
  def generate(self, enc_hs, start_id, sample_size): # enc_hs: Encoder의 출력(은닉 상태), start_id: 최초로 주어지는 단어 ID, sample_size: 샘플링하는 단어의 수
    sampled = []
    sample_id = start_id
    h = enc_hs[:, -1]
    self.lstm.set_state(h)

    for _ in range(sample_size):
        x = np.array([sample_id]).reshape((1, 1))

        out = self.embed.forward(x)
        dec_hs = self.lstm.forward(out)
        c = self.attention.forward(enc_hs, dec_hs)
        out = np.concatenate((c, dec_hs), axis=2)
        score = self.affine.forward(out)

        sample_id = np.argmax(score.flatten())
        sampled.append(sample_id)

    return sampled

### AttentionSeq2seq 구현    

In [12]:
class AttentionSeq2seq(Seq2seq):
  def __init__(self, vocab_size, wordvec_size, hidden_size):
    V, D, H = vocab_size, wordvec_size, hidden_size
    self.encoder = AttentionEncoder(V, D, H)
    self.decoder = AttentionDecoder(V, D, H)
    self.softmax = TimeSoftmaxWithLoss()

    self.params = self.encoder.params + self.decoder.params
    self.grads = self.encoder.grads + self.decoder.grads

### AttentionSeq2seq 학습     

In [13]:
from sequence import *
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq


In [14]:
# 데이터셋 읽기
(x_train, t_train), (x_test, t_test) = load_data('date.txt')
char_to_id, id_to_char = get_vocab()

# 입력 문장 반전
x_train, x_test = x_train[:, ::-1], x_test[:, ::-1]

# 하이퍼파라미터 설정
vocab_size = len(char_to_id)
wordvec_size = 16
hidden_size = 256
batch_size = 128
max_epoch = 1
max_grad = 5.0

model = AttentionSeq2seq(vocab_size, wordvec_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

In [15]:
acc_list = []

for epoch in range(max_epoch):
  trainer.fit(x_train, t_train, max_epoch=1, batch_size=batch_size, max_grad=max_grad)
  correct_num = 0
  for i in range(len(x_test)):
    question, correct = x_test[[i]], t_test[[i]]
    # 
    verbose = i < 10
    correct_num += eval_seq2seq(model, question, correct, id_to_char, verbose, is_reverse=True) # is_reverse=True로 지정하여 입력 문장을 반전시킨다.
    
  acc = float(correct_num) / len(x_test)
  acc_list.append(acc)
  print('정확도', acc)
    

| 에폭 1 |  반복 1 / 351 | 시간 0[s] | 손실 4.08
| 에폭 1 |  반복 21 / 351 | 시간 8[s] | 손실 3.09
| 에폭 1 |  반복 41 / 351 | 시간 16[s] | 손실 1.90
| 에폭 1 |  반복 61 / 351 | 시간 24[s] | 손실 1.72
| 에폭 1 |  반복 81 / 351 | 시간 31[s] | 손실 1.46
| 에폭 1 |  반복 101 / 351 | 시간 39[s] | 손실 1.19
| 에폭 1 |  반복 121 / 351 | 시간 47[s] | 손실 1.14
| 에폭 1 |  반복 141 / 351 | 시간 55[s] | 손실 1.09
| 에폭 1 |  반복 161 / 351 | 시간 64[s] | 손실 1.06
| 에폭 1 |  반복 181 / 351 | 시간 71[s] | 손실 1.04
| 에폭 1 |  반복 201 / 351 | 시간 80[s] | 손실 1.03
| 에폭 1 |  반복 221 / 351 | 시간 88[s] | 손실 1.02
| 에폭 1 |  반복 241 / 351 | 시간 96[s] | 손실 1.02
| 에폭 1 |  반복 261 / 351 | 시간 103[s] | 손실 1.01
| 에폭 1 |  반복 281 / 351 | 시간 110[s] | 손실 1.00
| 에폭 1 |  반복 301 / 351 | 시간 118[s] | 손실 1.00
| 에폭 1 |  반복 321 / 351 | 시간 125[s] | 손실 1.00
| 에폭 1 |  반복 341 / 351 | 시간 133[s] | 손실 1.00
Q 10/15/94                     
T 1994-10-15
X 1978-08-11
---
Q thursday, november 13, 2008  
T 2008-11-13
X 1978-08-11
---
Q Mar 25, 2003                 
T 2003-03-25
X 1978-08-11
---
Q Tuesday, November 22, 2

In [16]:
# 파라미터 저장
model.save_params()
